In [0]:
import dlt
from pyspark.sql.functions import col, when, lower

# 🥇 Table source (bronze)
@dlt.view
def bronze_qualite_eau():
    return spark.read.table("bronze.bronze_plv")

# 🥈 Table transformée (silver)
@dlt.table(
    name="silver_qualite_eau_conformite",
    comment="Ajout de la colonne 'conformite' dans la table silver à partir de la bronze.bronze_plv",
    partition_cols=["cddept", "annee"],
    table_properties={
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_qualite_eau_conformite():
    df = dlt.read("lnd_databricks_workspace.bronze.bronze_plv")
    df_conforme = df.withColumn(
        "conformite",
        when(lower(col("conclusionprel")).like("%non conforme%"), "non").otherwise("oui")
    )
    return df_conforme
